In [176]:
from sklearn.preprocessing import LabelEncoder
import torch
import torchvision
import PIL
import numpy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import torch.nn.functional as F
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using:', device)

Using: cuda


In [177]:
# Pre processing and encoding of labels
label_encoder = LabelEncoder()

train_labels = pd.read_csv('train.csv', delimiter='\t')
test_labels = pd.read_csv('test.csv', delimiter='\t')
all_labels = pd.concat([train_labels['label'], test_labels['label']])
label_encoder.fit(all_labels)
train_labels['label'] = label_encoder.transform(train_labels['label'])
test_labels['label'] = label_encoder.transform(test_labels['label'])

In [178]:
class FashionDataset(Dataset):
    def __init__(self, root, labels=None, transform=None):
        self.labels = labels
        self.img_path = os.path.join(root, 'archive/images/')
        self.transform = transform

    
    def __len__(self):
        return len(self.labels)
    

    def __getitem__(self, idx):
        label = self.labels.iloc[idx, 1]
        img_name = self.labels.iloc[idx, 0]
        img = PIL.Image.open(self.img_path + str(img_name) + '.jpg')
        
        if img.mode != 'RGB':
            img = img.convert('RGB')

        if self.transform:
            img = self.transform(img)
        return img, label

# The model of choice


In [179]:
class CNN(nn.Module):
    def __init__(self, num_classes=13):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(32 * 32 * 32, 160)
        self.fc2 = nn.Linear(160, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        print(out.shape)
        out = out.view(-1, 32 * 32 * 32)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

    

# Part 1:
Hyper Parameter Initialization

In [180]:
lr = 0.001
num_epochs = 5
batch_size = 25
momentum = 0.9
optim = 'adam'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # image net mean and std
])

Training

In [181]:
# Initialize the Dataset and DataLoader
print ('Initializing Dataset and DataLoader')
train_dataset = FashionDataset('', labels=train_labels, transform=transform)
test_dataset = FashionDataset('', labels=test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print ('Initializing Model')

model = CNN().to(device)


criterion = nn.CrossEntropyLoss()
if optim == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
if optim == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

best_acc = 0

print ('Training Model')
for epoch in range(num_epochs):

    running_loss = 0.0
    running_acc = 0.0

    print(f'Epoch {epoch + 1}/{num_epochs}')
    model.train()
    for i, (img, label) in enumerate(tqdm(train_loader, total=len(train_loader))):
        img = img.to(device)
        label = label.to(device).long()

        optimizer.zero_grad()
        output = model(img)

        loss = criterion(output, label)

        running_loss += loss.item() * img.size(0)

        loss.backward()
        optimizer.step()

        
    model.eval()
    correct = 0
    with torch.no_grad():
        for img, label in tqdm(test_loader):
            img = img.to(device)
            label = label.to(device).long()
            output = model(img)
            _, preds = torch.max(output, 1)

            pred_first = preds[0].cpu().numpy()
            label_first = label[0].cpu().numpy()

            correct += torch.sum(preds == label.data)
     


    acc =  correct.double() / len(test_dataset)
    print(f'Accuracy: {acc}')
    if acc > best_acc:
        best_acc = acc
        print (f'Saving best model')
        torch.save(model.state_dict(), 'best_model.pth')
        

Initializing Dataset and DataLoader
Initializing Model
Training Model
Epoch 1/5


  0%|          | 0/1618 [00:00<?, ?it/s]

torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25, 32, 32, 32])
torch.Size([25

KeyboardInterrupt: 

# Part 2
New Hyper Parameters and Data Augmentation